## Feature Selection

Vamos a seleccionar características mediante una combinación de varianza, eliminación de duplicados y de correlación. Al final, compararemos el rendimiento de los modelos de aprendizaje automático creados con los diferentes subconjuntos de características.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv(r'C:\Users\TomasCammisa\Desktop\Udemy\Feature Selection for ML\Precleaned datasets\dataset_1.csv')
data.shape

(50000, 301)

In [3]:
# Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels=['target'], axis=1),
    data['target'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((35000, 300), (15000, 300))

In [4]:
# Guardo una copia del conjunto de datos con todas las variables
# para medir el rendimiento de los modelos de aprendizaje automático

X_train_original = X_train.copy()
X_test_original = X_test.copy()

### Removemos variables constantes

Podemos hacer de dos maneras:

In [5]:
#La primera manera es calculando la desviación estandar de cada feature
#Obtenemos las que tiene std = 0 y las eliminamos de nuestro dataset
constant_features = [
    feat for feat in X_train.columns if X_train[feat].std() == 0
]

X_train.drop(labels=constant_features, axis=1, inplace=True)
X_test.drop(labels=constant_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 266), (15000, 266))

In [6]:
# El camino alternativo es la utilización de VarianceThreshold de scikit learn

sel = VarianceThreshold(     #Definimos umbral minimo de varianza 
    threshold=0)  

sel.fit(X_train)  # fit encuentra variables con varianza inferior al minimo

sum(sel.get_support()) # Contamos cantidad de variables que superan el umbral minimo

266

In [7]:
#Guardamos variables con las que nos quedamos
features_to_keep = X_train.columns[sel.get_support()] 

In [8]:
features_to_keep

Index(['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8',
       'var_9', 'var_10',
       ...
       'var_289', 'var_290', 'var_291', 'var_292', 'var_293', 'var_295',
       'var_296', 'var_298', 'var_299', 'var_300'],
      dtype='object', length=266)

In [9]:
# Removemos las variables
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train.shape, X_test.shape

((35000, 266), (15000, 266))

In [10]:
# Transformarmos el array que obtenemos como resultado en un nuevo df con las variables 
#que superaron el umbral minimo de varianza

X_train= pd.DataFrame(X_train)
X_train.columns = features_to_keep

X_test= pd.DataFrame(X_test)
X_test.columns = features_to_keep

In [11]:
X_train.shape

(35000, 266)

### Removemos variables cuasi constantes

El minimo de varianza es subjetivo al problema que estemos resolviendo. 

In [6]:
# eliminamos variables cuasi constantes
sel = VarianceThreshold(
    threshold=0.01)  

sel.fit(X_train)  # fit encuentra variables con varianza inferior al minimo

sum(sel.get_support()) # Contamos cantidad de variables que superan el umbral minimo

215

In [7]:
#Guardamos variables con las que nos quedamos
features_to_keep = X_train.columns[sel.get_support()] 

In [8]:
features_to_keep

Index(['var_3', 'var_4', 'var_5', 'var_6', 'var_8', 'var_11', 'var_12',
       'var_13', 'var_14', 'var_15',
       ...
       'var_286', 'var_288', 'var_290', 'var_291', 'var_292', 'var_293',
       'var_295', 'var_296', 'var_299', 'var_300'],
      dtype='object', length=215)

In [9]:
# Removemos las variables
X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train.shape, X_test.shape

((35000, 215), (15000, 215))

In [10]:
# Transformarmos el array que obtenemos como resultado en un nuevo df con las variables 
#que superaron el umbral minimo de varianza

X_train= pd.DataFrame(X_train)
X_train.columns = features_to_keep

X_test= pd.DataFrame(X_test)
X_test.columns = features_to_keep

In [11]:
X_train.head()

,var_3,var_4,var_5,var_6,var_8,var_11,var_12,var_13,var_14,var_15,...,var_286,var_288,var_290,var_291,var_292,var_293,var_295,var_296,var_299,var_300
0,0.0,2.79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,2.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.79,85435.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,5.70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Ya eliminamos 75 variables de nuestro data set de 300, es decir, el 25%

En caso de contar con modulos de informacion distinta, donde las unidades de medida de cada feature son distintas es recomendable:
1. Subdividir en modulos y aplicar el minimo de varianza a cada uno de ellos o
2. Normalizar el dataset antes de aplicar el minimo de varianza

### Removemos features duplicadas

Los datasets suelen features duplicadas, es decir, features que, a pesar de tener nombres diferentes, son idénticas. Muchas veces incluimos features duplicadas cuando realizamos una codificación de variables categóricas.

No hay función en Pandas para encontrar columnas duplicadas,por lo que debemos crear un codigo para iterar sobre nuestra base de datos y eliminar las mismas.

Advetencia: puede ser una operación computacionalmente costosa en Python, por lo tanto, dependiendo del tamaño de su conjunto de datos, es posible que no siempre pueda hacerlo.



1. Codigo detallado

In [18]:
# comprobar si hay features duplicadas en el dataset:

# crear un diccionario donde almacenaremos
# las features duplicadas
duplicated_feat_pairs = {}


# creamos una lista vacía para alamcenar las features
# que se encontraron duplicados
_duplicated_feat = []


# iteramos sobre cada feature en nuestro dataset:
for i in range(0, len(X_train.columns)):
    
    # esta linea de codigo nos ayuda a entender dónde está el loop
    if i % 10 == 0:  
        print(i)
    
    # seleccionamos la feature 1:
    feat_1 = X_train.columns[i]
    
    # verifica si esta feature ya ha sido identificada
    # como un duplicado de otra. Si lo fuera, debe almacenarse en
    # nuestra lista _duplicated_feat.
    
    # Si esta función ya se identificó como un duplicado, la omitimos, si
    # aún no ha sido identificado como duplicado, entonces seguimos:
    if feat_1 not in _duplicated_feat:
    
        # creamos una lista vacía como entrada para esta función en el diccionario:
        duplicated_feat_pairs[feat_1] = []

        # iteramos sobre el resto de las columnas del dataset:
        for feat_2 in X_train.columns[i + 1:]:

            # comprobamose si esta segunda feature es idéntica a la primera
            if X_train[feat_1].equals(X_train[feat_2]):

                # si es idéntica, la agregamos al diccionaro
                duplicated_feat_pairs[feat_1].append(feat_2)
                
                # y la sumamos a nuestra lista de variables duplicadas
                _duplicated_feat.append(feat_2)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


In [19]:
# Verificamos extension de nuestra lista de variables duplicadas
len(_duplicated_feat)

10

In [20]:
# Estas variables son:
_duplicated_feat

['var_151',
 'var_183',
 'var_148',
 'var_216',
 'var_199',
 'var_296',
 'var_239',
 'var_263',
 'var_232',
 'var_269']

In [21]:
# Veamos el diccionario de pares de features duplicadas
# Si, por cada variable, tiene una feature en la lista, quiere decir que esta feature esta duplicada.
duplicated_feat_pairs

{'var_3': [],
 'var_4': [],
 'var_5': [],
 'var_6': ['var_151'],
 'var_8': [],
 'var_11': [],
 'var_12': [],
 'var_13': [],
 'var_14': [],
 'var_15': [],
 'var_16': [],
 'var_17': [],
 'var_18': [],
 'var_20': [],
 'var_21': [],
 'var_22': [],
 'var_24': [],
 'var_25': [],
 'var_26': [],
 'var_27': [],
 'var_29': [],
 'var_30': [],
 'var_31': [],
 'var_32': [],
 'var_34': ['var_183'],
 'var_35': [],
 'var_37': ['var_148'],
 'var_38': [],
 'var_39': [],
 'var_40': [],
 'var_41': [],
 'var_42': [],
 'var_46': [],
 'var_47': [],
 'var_48': [],
 'var_49': [],
 'var_50': [],
 'var_51': [],
 'var_52': [],
 'var_54': [],
 'var_55': [],
 'var_57': [],
 'var_58': [],
 'var_60': ['var_216'],
 'var_62': [],
 'var_63': [],
 'var_64': [],
 'var_65': [],
 'var_68': [],
 'var_70': [],
 'var_72': [],
 'var_73': [],
 'var_74': [],
 'var_75': [],
 'var_76': [],
 'var_77': [],
 'var_78': [],
 'var_79': [],
 'var_82': [],
 'var_83': [],
 'var_84': ['var_199'],
 'var_85': [],
 'var_86': [],
 'var_88': [],


In [22]:
# Veamos solo las features duplicadas

# iteramos sobre cada variable en el diccionario:
for feat in duplicated_feat_pairs.keys():
    
    # si tiene duplicados, la lista no debe estar vacía::
    if len(duplicated_feat_pairs[feat]) > 0:

        # hacemos print de la varaible y su duplicado:
        print(feat, duplicated_feat_pairs[feat])
        print()

var_6 ['var_151']

var_34 ['var_183']

var_37 ['var_148']

var_60 ['var_216']

var_84 ['var_199']

var_143 ['var_296']

var_149 ['var_239']

var_221 ['var_263']

var_226 ['var_232']

var_229 ['var_269']



In [23]:
# Seleccionemos un par de features para ver de que se trata

X_train[['var_6', 'var_151']].head(10)

,var_6,var_151
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,0.0,0.0
9,0.0,0.0


2. Codigo simplificado

In [12]:
# Buscamos variables dupplicadas en nuestro dataset
duplicated_feat = []  #Creamos una lista donde alamacenaremos las variables duplicadas
for i in range(0, len(X_train.columns)): #iteramos sobre nuestro df
    if i % 10 == 0:  
        print(i)

    col_1 = X_train.columns[i] #Seleccionamos la primer feature

    for col_2 in X_train.columns[i + 1:]: 
        if X_train[col_1].equals(X_train[col_2]):
            duplicated_feat.append(col_2)
            
len(duplicated_feat)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210


10

In [13]:
# removemos duplicados
X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
X_test.drop(labels=duplicated_feat, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 205), (15000, 205))

In [14]:
# Guardamos una copia del df
# para medir el rendimiento de los modelos
# al final

X_train_basic_filter = X_train.copy()
X_test_basic_filter = X_test.copy()

### Eliminamos variables correlacionadas

¿Porque eliminar variables correlacionadas?
"Los buenos subconjuntos de datasets contienen variables altamente correlacionadas con la variable objetivo, pero no correlacionadas entre sí". (Cammisa, 2022, p.271)

In [17]:
# con la siguiente función podemos seleccionar features altamente correlacionadas
# eliminará la primera feature que se correlaciona con cualquiera otra
# sin más información.

def correlation(dataset, threshold):
    col_corr = set()  # creamos un set donde guardamos los nombres de las columnas correlacionadas
    
    corr_matrix = dataset.corr() #creamos la matriz de correlacion
    
    # para cada feature en el dataset (columnas de la matriz de correlación)
    for i in range(len(corr_matrix.columns)):  
    
        for j in range(i):  #verificamos con otras features
            
            # si la correlación es superior a un cierto umbral
            if abs(corr_matrix.iloc[i, j]) > threshold:
                
                #linea opcional para ver correlacion entre cada par de variables con corr mayor al umbral minimo
                print(abs(corr_matrix.iloc[i, j]), corr_matrix.columns[i], corr_matrix.columns[j]) 
                
                colname = corr_matrix.columns[i]  # obtenemos el nombre de las columnas correlacionadas
                col_corr.add(colname)  #lo agregamos a nuestro set de variables correlacionadas
    return col_corr

corr_features = correlation(X_train, 0.8) #llamamos a la funcion
print('correlated features: ', len(set(corr_features)) )

0.9901870592788291 var_14 var_13
0.9747684936252511 var_40 var_16
0.9019247145301009 var_54 var_51
0.8080394007343367 var_55 var_21
0.9469585731168426 var_62 var_31
0.9871515157796921 var_64 var_47
0.8790256734547003 var_65 var_3
1.0000000000000098 var_73 var_34
0.9215085514013303 var_86 var_30
0.9996086272168176 var_88 var_38
0.840948382976813 var_93 var_41
0.992979818656033 var_94 var_84
0.999171276440353 var_100 var_96
0.9364005277628559 var_101 var_18
0.8473817067229984 var_102 var_16
0.884479402585018 var_102 var_40
0.8906887043129583 var_103 var_52
0.8842323791033176 var_105 var_91
0.8919991045566614 var_107 var_58
0.9798048008794173 var_114 var_58
0.8691881944783044 var_114 var_107
0.9061739769575459 var_117 var_38
0.8017092998502472 var_117 var_58
0.9065287676443693 var_117 var_88
0.8039746471057304 var_117 var_114
0.9294747773002267 var_121 var_83
0.9048991262430333 var_123 var_52
0.9864956619134007 var_123 var_103
0.9548493309404984 var_125 var_98
0.9901870592788358 var_129 v

In [15]:
#Codigo simplificado
# con la siguiente función podemos seleccionar features altamente correlacionadas
# eliminará la primera feature que se correlaciona con cualquiera otra
# sin más información.

def correlation(dataset, threshold):
    col_corr = set()  # creamos un set donde guardamos los nombres de las columnas correlacionadas
    
    corr_matrix = dataset.corr() #creamos la matriz de correlacion
    
    # para cada feature en el dataset (columnas de la matriz de correlación)
    for i in range(len(corr_matrix.columns)):  
    
        for j in range(i):  #verificamos con otras features
            
            # si la correlación es superior a un cierto umbral
            if abs(corr_matrix.iloc[i, j]) > threshold:
                
                #linea opcional para ver correlacion entre cada par de variables con corr mayor al umbral minimo
                #print(abs(corr_matrix.iloc[i, j]), corr_matrix.columns[i], corr_matrix.columns[j]) 
                
                colname = corr_matrix.columns[i]  # obtenemos el nombre de las columnas correlacionadas
                col_corr.add(colname)  #lo agregamos a nuestro set de variables correlacionadas
    return col_corr

corr_features = correlation(X_train, 0.8) #llamamos a la funcion
print('correlated features: ', len(set(corr_features)) )

correlated features:  93


In [16]:
X_train.drop(labels=corr_features, axis=1, inplace=True)
X_test.drop(labels=corr_features, axis=1, inplace=True)

X_train.shape, X_test.shape

((35000, 112), (15000, 112))

### Comparamos performance de los modelos

In [17]:
# crear una función para random frest y comparar el rendimiento en el conjunto de entrenamiento y prueba

def run_randomForests(X_train, X_test, y_train, y_test):
    rf = RandomForestClassifier(n_estimators=200, random_state=39, max_depth=4)
    rf.fit(X_train, y_train)
    print('Train set')
    pred = rf.predict_proba(X_train)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = rf.predict_proba(X_test)
    print('Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [18]:
# df original con todas las variables
run_randomForests(X_train_original,
                  X_test_original,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.807612232524249
Test set
Random Forests roc-auc: 0.7868832427636059


In [19]:
# filter methods - Basico - VARIANZA Y VARIABLES DUPLICADAS
run_randomForests(X_train_basic_filter,
                  X_test_basic_filter,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.810290026780428
Test set
Random Forests roc-auc: 0.7914020645941601


In [20]:
# filter methods - Correlación
run_randomForests(X_train,
                  X_test,
                  y_train, y_test)

Train set
Random Forests roc-auc: 0.8066004772684517
Test set
Random Forests roc-auc: 0.7859521124929707


La eliminación de variables constantes, casi constantes, duplicadas y correlacionadas redujo la cantidad de features drasticamente (de 300 a 112), sin afectar el rendimiento de los distintos modelos (0,786 frente a 0,7859).

In [21]:
# creamos una función para construir una regresión logística y comparar el rendimiento

def run_logistic(X_train, X_test, y_train, y_test):
    # función para entrenar y probar el rendimiento de la regresión logística
    logit = LogisticRegression(random_state=44, max_iter=500)
    logit.fit(X_train, y_train)
    print('Train set')
    pred = logit.predict_proba(X_train)
    print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:,1])))
    print('Test set')
    pred = logit.predict_proba(X_test)
    print('Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:,1])))

In [22]:
# original
# Primero escalamos los datos

# original
scaler = StandardScaler().fit(X_train_original)

run_logistic(scaler.transform(X_train_original),
             scaler.transform(X_test_original), y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8028231106726094
Test set
Logistic Regression roc-auc: 0.7950984398269426


In [23]:
# filter methods - Basico
scaler = StandardScaler().fit(X_train_basic_filter)

run_logistic(scaler.transform(X_train_basic_filter),
             scaler.transform(X_test_basic_filter),
                  y_train, y_test)

Train set
Logistic Regression roc-auc: 0.8022733407131674
Test set
Logistic Regression roc-auc: 0.7947416996589153


In [24]:
# filter methods - Correlación
scaler = StandardScaler().fit(X_train)

run_logistic(scaler.transform(X_train),
             scaler.transform(X_test),
                  y_train, y_test)

Train set
Logistic Regression roc-auc: 0.7942679586909129
Test set
Logistic Regression roc-auc: 0.7881800430100233


De manera similar, para la regresión logística, la eliminación de características constantes, casi constantes, duplicadas y altamente correlacionadas no afectó drásticamente el rendimiento del algoritmo.

## Tarea
1. Probar distintos umbrales de varianza minima y ver como afecta a los resultados
2. Probar con distintos umbrales de correlacion y ver como afecta a los modelos
2. Probar con distinto set de datos